In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hinditotelugu/FINAL_ML (1).xlsx


In [2]:
# Install openpyxl to read Excel files
!pip install openpyxl

# Step 1: Import Necessary Libraries

import numpy as np
import pandas as pd
import os
import string
from string import digits
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf

print("Libraries imported successfully.")



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


E0000 00:00:1732688214.627702   16723 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1127 06:16:54.636146419   16723 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1127 06:16:54.636160960   16723 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1127 06:16:54.636164135   16723 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1127 06:16:54.636166467   16723 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1127 06:16:54.636168759   16723 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

Libraries imported successfully.


In [3]:
# Step 2: Set up TPU

try:
    # Detect TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    # Connect to TPU cluster
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # Create TPU strategy
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()  # Default strategy

print("REPLICAS: ", strategy.num_replicas_in_sync)


Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1732688219.565248   16723 service.cc:145] XLA service 0x587afa8c17a0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732688219.565321   16723 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1732688219.565326   16723 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1732688219.565329   16723 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1732688219.565332   16723 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1732688219.565335   16723 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1732688219.565355   16723 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1732688219.565358   16723 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1732688219.565360   16723 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
# Step 3: Load the Dataset

# List files in the input directory
print("Files in the input directory:")
print(os.listdir("../input"))

# Adjust the file path according to your dataset
# Replace 'hinditotelugu' and 'FINAL_ML (1).xlsx' with actual names if different
data = pd.read_excel("/kaggle/input/hinditotelugu/FINAL_ML (1).xlsx", names=['HINDI', 'TELUGU'], header=None, skiprows=1)

# Display the first few rows
print("\nFirst few rows of the dataset:")
print(data.head())


Files in the input directory:
['hinditotelugu']

First few rows of the dataset:
                                  HINDI  \
0                                शीर्षक   
1             आईडीबीआई पर आरबीआई की नजर   
2  आज बैंकिंग प्रमुखों से मिलेंगे जेटली   
3    जडेजा ने एक महत्वपूर्ण विकेट लिया।   
4    पाकिस्तान की एक और भड़काऊ कार्रवाई   

                                    TELUGU  
0                                  శీర్షిక  
1                   ఐడిబిఐపై ఆర్‌బిఐ నజర్‌  
2    బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ  
3                కీలక వికెట్ తీసిన జడేజా..  
4  మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్  


In [5]:
# Step 4: Preprocess the Data

# Check for missing values
print("\nNumber of missing values in HINDI column:", data['HINDI'].isnull().sum())
print("Number of missing values in TELUGU column:", data['TELUGU'].isnull().sum())

# Drop rows with missing values
data = data.dropna(subset=['HINDI', 'TELUGU'])
print("After dropping missing values, data shape:", data.shape)

# Remove empty strings
data = data[data['HINDI'].str.strip().astype(bool)]
data = data[data['TELUGU'].str.strip().astype(bool)]
data.reset_index(drop=True, inplace=True)

# Convert all entries to strings
data['HINDI'] = data['HINDI'].astype(str)
data['TELUGU'] = data['TELUGU'].astype(str)

# Lowercase all characters
data['HINDI'] = data['HINDI'].apply(lambda x: x.lower())
data['TELUGU'] = data['TELUGU'].apply(lambda x: x.lower())

# Remove quotes
data['HINDI'] = data['HINDI'].apply(lambda x: x.replace("'", ''))
data['TELUGU'] = data['TELUGU'].apply(lambda x: x.replace("'", ''))

# Remove punctuation
exclude = set(string.punctuation)
data['HINDI'] = data['HINDI'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data['TELUGU'] = data['TELUGU'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove digits
remove_digits = str.maketrans('', '', digits)
data['HINDI'] = data['HINDI'].apply(lambda x: x.translate(remove_digits))
data['TELUGU'] = data['TELUGU'].apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
data['HINDI'] = data['HINDI'].apply(lambda x: x.strip())
data['TELUGU'] = data['TELUGU'].apply(lambda x: x.strip())
data['HINDI'] = data['HINDI'].apply(lambda x: re.sub(" +", " ", x))
data['TELUGU'] = data['TELUGU'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
data['TELUGU'] = data['TELUGU'].apply(lambda x: 'START_ ' + x + ' _END')

# Display the preprocessed data
print("\nPreprocessed Data:")
print(data.head())



Number of missing values in HINDI column: 1
Number of missing values in TELUGU column: 0
After dropping missing values, data shape: (21403, 2)

Preprocessed Data:
                                  HINDI  \
0                                शीर्षक   
1             आईडीबीआई पर आरबीआई की नजर   
2  आज बैंकिंग प्रमुखों से मिलेंगे जेटली   
3    जडेजा ने एक महत्वपूर्ण विकेट लिया।   
4    पाकिस्तान की एक और भड़काऊ कार्रवाई   

                                              TELUGU  
0                                START_ శీర్షిక _END  
1                 START_ ఐడిబిఐపై ఆర్‌బిఐ నజర్‌ _END  
2  START_ బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ _END  
3                START_ కీలక వికెట్ తీసిన జడేజా _END  
4  START_ మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్...  


In [6]:
# Step 5: Create Vocabulary Dictionaries

all_hindi_words = set()
for hin in data['HINDI']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

all_telugu_words = set()
for tel in data['TELUGU']:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

print("\nTotal unique Hindi words:", len(all_hindi_words))
print("Total unique Telugu words:", len(all_telugu_words))



Total unique Hindi words: 16068
Total unique Telugu words: 32316


In [7]:
# Step 6: Calculate Sentence Lengths and Filter Sentences

data['length_hindi_sentence'] = data['HINDI'].apply(lambda x: len(x.split(" ")))
data['length_telugu_sentence'] = data['TELUGU'].apply(lambda x: len(x.split(" ")))

print("\nSentence length statistics:")
print(data[['length_hindi_sentence', 'length_telugu_sentence']].describe())

# Set maximum sentence lengths (adjust as needed)
max_length_src = 20  # Hindi sentences
max_length_tar = 20  # Telugu sentences

# Filter sentences based on length
data = data[data['length_hindi_sentence'] <= max_length_src]
data = data[data['length_telugu_sentence'] <= max_length_tar]
data.reset_index(drop=True, inplace=True)

print("\nData after filtering based on sentence lengths:")
print(data.head())
print("Total samples after filtering:", data.shape[0])



Sentence length statistics:
       length_hindi_sentence  length_telugu_sentence
count           21403.000000            21403.000000
mean                7.079615                6.328272
std                16.366744                1.589870
min                 1.000000                3.000000
25%                 5.000000                5.000000
50%                 7.000000                6.000000
75%                 9.000000                7.000000
max              2361.000000               14.000000

Data after filtering based on sentence lengths:
                                  HINDI  \
0                                शीर्षक   
1             आईडीबीआई पर आरबीआई की नजर   
2  आज बैंकिंग प्रमुखों से मिलेंगे जेटली   
3    जडेजा ने एक महत्वपूर्ण विकेट लिया।   
4    पाकिस्तान की एक और भड़काऊ कार्रवाई   

                                              TELUGU  length_hindi_sentence  \
0                                START_ శీర్షిక _END                      1   
1                 START_ ఐడి

In [8]:
# Step 7: Prepare Data for Training

# Sort the unique words to maintain consistency
input_words = sorted(list(all_hindi_words))
target_words = sorted(list(all_telugu_words))

# Define the number of unique tokens (+1 for padding)
num_encoder_tokens = len(input_words) + 1  # +1 for padding token
num_decoder_tokens = len(target_words) + 1  # +1 for padding token

# Create word-to-index and index-to-word dictionaries for Hindi
input_token_index = dict([(word, i + 1) for i, word in enumerate(input_words)])
reverse_input_token_index = dict((i, word) for word, i in input_token_index.items())

# Create word-to-index and index-to-word dictionaries for Telugu
target_token_index = dict([(word, i + 1) for i, word in enumerate(target_words)])
reverse_target_token_index = dict((i, word) for word, i in target_token_index.items())

print("\nNumber of unique input tokens (Hindi):", num_encoder_tokens)
print("Number of unique output tokens (Telugu):", num_decoder_tokens)



Number of unique input tokens (Hindi): 16069
Number of unique output tokens (Telugu): 32317


In [9]:
# Step 8: Prepare Sequences and Split the Data

# Prepare input sequences
encoder_input_data = []
for input_text in data['HINDI']:
    encoder_input_data.append([input_token_index.get(word, 0) for word in input_text.split()])

# Prepare decoder input and output sequences
decoder_input_data = []
decoder_target_data = []
for target_text in data['TELUGU']:
    target_words = target_text.split()
    decoder_input_data.append([target_token_index.get(word, 0) for word in target_words[:-1]])  # exclude _END token
    decoder_target_data.append([target_token_index.get(word, 0) for word in target_words[1:]])  # exclude START_ token

# Pad sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_length_src, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_length_tar, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_length_tar, padding='post')

# Convert decoder_target_data to one-hot vectors
decoder_target_data_onehot = tf.keras.utils.to_categorical(decoder_target_data, num_classes=num_decoder_tokens)

print("\nData prepared for training.")

# Split the data into training and test sets
X_train_enc, X_test_enc, X_train_dec_inp, X_test_dec_inp, y_train, y_test = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data_onehot, test_size=0.2, random_state=42)

print("\nTraining and test data sizes:")
print("Training data:", X_train_enc.shape)
print("Test data:", X_test_enc.shape)



Data prepared for training.

Training and test data sizes:
Training data: (17112, 20)
Test data: (4278, 20)


In [10]:
# Step 9: Build the Encoder-Decoder Model

latent_dim = 256  # Adjust as needed

with strategy.scope():
    # Encoder
    encoder_inputs = tf.keras.layers.Input(shape=(None,), name='encoder_inputs')
    enc_emb = tf.keras.layers.Embedding(num_encoder_tokens, latent_dim, mask_zero=True, name='encoder_embedding')(encoder_inputs)
    encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(latent_dim, return_state=True, name='encoder_lstm')(enc_emb)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = tf.keras.layers.Input(shape=(None,), name='decoder_inputs')
    dec_emb_layer = tf.keras.layers.Embedding(num_decoder_tokens, latent_dim, mask_zero=True, name='decoder_embedding')
    dec_emb = dec_emb_layer(decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
    decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

print("\nModel Summary:")
model.summary()


I0000 00:00:1732688296.173942   16723 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 256) │  4,113,664 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 256) │  8,273,152 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    525,312 │ encoder_embeddin… │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    525,312 │ decoder_embeddin… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None,      │  8,305,469 │ decoder_lstm[0][… │
│ (Dense)             │ 32317)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,742,909 (82.94 MB)

 Trainable params: 21,742,909 (82.94 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Step 10: Train the Model

batch_size = 64  # Adjust as needed (should be a multiple of 8 for TPU)
epochs = 50  # Adjust as needed

# Since we are using numpy arrays, we can use model.fit directly
history = model.fit([X_train_enc, X_train_dec_inp], y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([X_test_enc, X_test_dec_inp], y_test))


Epoch 1/50


2024-11-27 06:21:26.121865: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1732688486.796878   17527 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(15f3aafa88e99087:0:0), session_name()


  2/268 ━━━━━━━━━━━━━━━━━━━━ 27s 103ms/step - accuracy: 0.0125 - loss: 10.3828    

I0000 00:00:1732688496.066927   17527 tpu_compile_op_common.cc:245] Compilation of 15f3aafa88e99087:0:0 with session name  took 9.269339593s and succeeded
I0000 00:00:1732688496.085307   17527 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(15f3aafa88e99087:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_2097779827705844746", property.function_library_fingerprint = 6845137501007871305, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "8,20,;8,20,;8,20,32317,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1732688496.085362   17527 tpu_compilation_cache_interface.cc:541] After adding entry for 

266/268 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0491 - loss: 8.5914

...

268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.0491 - loss: 8.5885

I0000 00:00:1732688521.633093   17532 tpu_compile_op_common.cc:245] Compilation of d8d6e9d2ef7625dc:0:0 with session name  took 7.646507455s and succeeded
I0000 00:00:1732688521.647568   17532 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d8d6e9d2ef7625dc:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_2097779827705844746", property.function_library_fingerprint = 6845137501007871305, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "3,20,;3,20,;3,20,32317,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1732688521.647613   17532 tpu_compilation_cache_interface.cc:541] After adding entry for 

268/268 ━━━━━━━━━━━━━━━━━━━━ 79s 252ms/step - accuracy: 0.0491 - loss: 8.5870 - val_accuracy: 0.0513 - val_loss: 8.0019
Epoch 2/50


I0000 00:00:1732688563.381017   17513 tpu_compile_op_common.cc:245] Compilation of bef76304a1e22860:0:0 with session name  took 1.726608478s and succeeded
I0000 00:00:1732688563.385427   17513 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(bef76304a1e22860:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_8410191941267949904", property.function_library_fingerprint = 6081821840311488523, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "7,20,;7,20,;7,20,32317,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1732688563.385455   17513 tpu_compilation_cache_interface.cc:541] After adding entry for 

268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 97ms/step - accuracy: 0.0512 - loss: 7.8613 - val_accuracy: 0.0515 - val_loss: 7.8803
Epoch 3/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 97ms/step - accuracy: 0.0514 - loss: 7.6784 - val_accuracy: 0.0518 - val_loss: 7.8026
Epoch 4/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 97ms/step - accuracy: 0.0529 - loss: 7.4875 - val_accuracy: 0.0530 - val_loss: 7.7799
Epoch 5/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 99ms/step - accuracy: 0.0540 - loss: 7.3180 - val_accuracy: 0.0543 - val_loss: 7.6354
Epoch 6/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 99ms/step - accuracy: 0.0561 - loss: 7.0557 - val_accuracy: 0.0553 - val_loss: 7.6062
Epoch 7/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 99ms/step - accuracy: 0.0571 - loss: 6.9220 - val_accuracy: 0.0557 - val_loss: 7.5101
Epoch 8/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 27s 98ms/step - accuracy: 0.0586 - loss: 6.7713 - val_accuracy: 0.0540 - val_loss: 7.5001
Epoch 9/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - accuracy: 0.0607 - loss: 6.5766 - val_accurac

In [12]:
# Step 11: Save the Model Weights

import tensorflow as tf

# Explicitly specify the CPU device for saving weights
with tf.device('/CPU:0'):
    model.save_weights('nmt_weights.weights.h5')
    print("\nModel weights saved as 'nmt_weights.weights.h5'.")


ValueError: Device /job:localhost/replica:0/task:0/device:CPU:0 is not found

In [13]:
# Step 12: Build the Inference Model

with strategy.scope():
    # Encoder inference model
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    # Decoder inference model
    decoder_state_input_h = tf.keras.layers.Input(shape=(latent_dim,), name='decoder_state_input_h')
    decoder_state_input_c = tf.keras.layers.Input(shape=(latent_dim,), name='decoder_state_input_c')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    dec_emb2 = dec_emb_layer(decoder_inputs)

    decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
    decoder_states2 = [state_h2, state_c2]
    decoder_outputs2 = decoder_dense(decoder_outputs2)

    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs2] + decoder_states2)

print("\nInference models are ready.")



Inference models are ready.


In [14]:
# Step 13: Define a Function to Decode Sequences

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token.
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index.get('START_', 0)

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_token_index.get(sampled_token_index, '')

        if sampled_word != '_END':
            decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop token.
        if (sampled_word == '_END' or len(decoded_sentence.split()) > max_length_tar):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

print("\nSequence decoding function is ready.")



Sequence decoding function is ready.


In [15]:
# Step 14: Test the Model with Sample Sentences

for seq_index in range(5):
    # Get the input sentence
    input_seq = X_test_enc[seq_index: seq_index + 1]

    # Decode the sequence to get the translated sentence
    decoded_sentence = decode_sequence(input_seq)

    # Get the actual TELUGU translation (remove START_ and _END tokens)
    actual_translation = data['TELUGU'][X_train_enc.shape[0] + seq_index][7:-5]  # Adjust indices if necessary

    print(f"\nInput Hindi sentence: {data['HINDI'][X_train_enc.shape[0] + seq_index]}")
    print(f"Actual Telugu Translation: {actual_translation}")
    print(f"Predicted Telugu Translation: {decoded_sentence}")


2024-11-27 06:46:12.378822: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_1_1/encoder_embedding_1/GatherV2/ReadVariableOp.
I0000 00:00:1732689972.472715   17468 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(1251be31d02d740d:0:0), session_name()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


I0000 00:00:1732689972.717095   17468 tpu_compile_op_common.cc:245] Compilation of 1251be31d02d740d:0:0 with session name  took 244.324371ms and succeeded
I0000 00:00:1732689972.718422   17468 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(1251be31d02d740d:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_5018147358608534046", property.function_library_fingerprint = 1710599553133815383, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,20,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1732689972.718447   17468 tpu_compilation_cache_interface.cc:541] After adding entry for key 1251be

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


I0000 00:00:1732689974.512396   17520 tpu_compile_op_common.cc:245] Compilation of 576d8e473c9da8c3:0:0 with session name  took 288.149959ms and succeeded
I0000 00:00:1732689974.514036   17520 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(576d8e473c9da8c3:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_7846581683446977397", property.function_library_fingerprint = 4117562564988594661, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,1,;2,256,;2,256,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1732689974.514082   17520 tpu_compilation_cache_interface.cc:541] After adding entry f

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step

Input Hindi sentence: श्रीलंकाई कप्तान नया है। दोनों की सारी उम्मीदें हैं
Actual Telugu Translation: లంక కెప్టెన్‌కూ కొత్తే ఆశలన్నీ వారిద్దరిపైనే
Predicted Telugu Translation: ఆ అమ్మడు మళ్లీ సుష్మా స్వరాజ్
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 976ms/step

Input Hindi sentence: श्वसन और हृदय रोगियों को हर समय आईसीयू की आवश्यकता नहीं होती है
Actual Telugu Translation: శ్వాసకోశ హృద్రోగులకు అన్ని సార్లు ఐసీయూ అక్కర్లేదు
Predicted Telugu Translation: వెంకీ సినిమాలో వెంకీ ఏంటి
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1